<a href="https://colab.research.google.com/github/sakuna47/Chatbot_with_NLP/blob/FrontEnd/CWNF_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import random
import json
import nltk

# Load intents.json
file_path = "intents.json"
with open(file_path) as file:
    data = json.load(file)

# Load trained GPT-2 Model
tokenizer = GPT2Tokenizer.from_pretrained("Chatbot_Model")
model = GPT2LMHeadModel.from_pretrained("Chatbot_Model")

# Title of the app
st.title("Chatbot with Rule-Based and Deep Learning Models")

# Sidebar to select chatbot type
chatbot_type = st.sidebar.radio("Select Chatbot Type", ["Rule-Based", "Deep Learning"])

# Initialize session state to store chat history
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

# Function to display chat history
def display_chat_history():
    for message in st.session_state.chat_history:
        st.write(f"**You:** {message['user']}")
        st.write(f"**Chatbot:** {message['bot']}")
        st.write("---")

# Rule-Based Chatbot Function
def get_response(user_input):
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            if pattern.lower() in user_input.lower():
                return random.choice(intent["responses"])
    return "I'm sorry, I don't understand that."

# Deep Learning Chatbot Function (GPT-2)
def generate_response(user_input):
    inputs = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors="pt")
    outputs = model.generate(inputs, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2, top_p=0.95, temperature=0.7)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Backend Function to Handle Chat
def chat_backend(user_input, chatbot_type="rule_based"):
    if chatbot_type == "Rule-Based":
        return get_response(user_input)
    elif chatbot_type == "Deep Learning":
        return generate_response(user_input)
    else:
        return "Invalid chatbot type selected."

# Chat input
user_input = st.text_input("You: ", "")

# When the user submits a message
if user_input:
    bot_response = chat_backend(user_input, chatbot_type)
    st.session_state.chat_history.append({"user": user_input, "bot": bot_response})
    st.text_input("You: ", "", key="input")

# Display chat history
display_chat_history()
